In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pylab as plt
#%pip install seaborn
import seaborn as sns

from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.compose import ColumnTransformer,TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder,PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator


from sklearn import set_config

# %pip install dagshub
# %pip install mlflow optuna
import mlflow
import mlflow.sklearn
import dagshub


from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor

import optuna


set_config(transform_output="pandas")
plt.style.use('fivethirtyeight')

In [7]:
dagshub.init(repo_owner='Arej02', repo_name='swiggy_delivery_time_prediction', mlflow=True)

Initialized MLflow to track repo "Arej02/swiggy_delivery_time_prediction"

Repository Arej02/swiggy_delivery_time_prediction initialized!

In [8]:
mlflow.set_tracking_uri("https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow")

In [9]:
mlflow.set_experiment("Exp3:Hyperparameter tuning")

<Experiment: artifact_location='mlflow-artifacts:/89e8f7ed08bc4de29a1e3ffa65e87006', creation_time=1758525702768, experiment_id='2', last_update_time=1758525702768, lifecycle_stage='active', name='Exp3:Hyperparameter tuning', tags={}>

In [10]:
swiggy=pd.read_csv("swiggy_cleaned.csv")

In [11]:
swiggy.sample(3)

,rider_id,age,ratings,restaurant_latitude,restaurant_longitude,delivery_latitude,delivery_longitude,order_date,weather,traffic,...,time_taken,city_name,order_day,order_month,order_day_of_week,is_weekend,pickup_time_minutes,order_time_hour,order_time_of_day,distance
6967,LUDHRES14DEL02,23.0,4.9,30.892978,75.821847,31.022978,75.951847,2022-02-18,sandstorms,jam,...,25,LUDH,18,2,friday,0,15.0,20.0,night,19.042586
38803,MUMRES04DEL03,39.0,4.6,19.254567,72.848923,19.344567,72.938923,2022-04-04,fog,jam,...,49,MUM,4,4,monday,0,10.0,19.0,evening,13.760880
40256,MYSRES12DEL02,24.0,4.8,12.284747,76.625861,12.414747,76.755861,2022-03-16,cloudy,low,...,24,MYS,16,3,wednesday,0,10.0,23.0,night,20.207798


In [12]:
swiggy['distance'].describe()

,distance
count,41872.000000
mean,9.719296
std,5.602890
min,1.465067
25%,4.657655
50%,9.193014
75%,13.680920
max,20.969489


In [13]:
def bin_distance(df:pd.DataFrame,num_col):
    distance_labels=['short','medium','long','very long']
    distance_bins=[1,6,12,18,24]
    df['distance_bins']=pd.cut(df[num_col],bins=distance_bins,labels=distance_labels,right=False)
    return df

In [14]:
bin_distance(swiggy,'distance')

,rider_id,age,ratings,restaurant_latitude,restaurant_longitude,delivery_latitude,delivery_longitude,order_date,weather,traffic,...,city_name,order_day,order_month,order_day_of_week,is_weekend,pickup_time_minutes,order_time_hour,order_time_of_day,distance,distance_bins
0,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,sunny,high,...,INDO,19,3,saturday,1,15.0,11.0,morning,3.025149,short
1,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,stormy,jam,...,BANG,25,3,friday,0,5.0,19.0,evening,20.183530,very long
2,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,sandstorms,low,...,BANG,19,3,saturday,1,15.0,8.0,morning,1.552758,short
3,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,sunny,medium,...,COIMB,5,4,tuesday,0,10.0,18.0,evening,7.790401,medium
4,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,cloudy,high,...,CHEN,26,3,saturday,1,15.0,13.0,afternoon,6.210138,medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45497,JAPRES04DEL01,30.0,4.8,26.902328,75.794257,26.912328,75.804257,2022-03-24,windy,high,...,JAP,24,3,thursday,0,10.0,11.0,morning,1.489846,short
45498,AGRRES16DEL01,21.0,4.6,NaN,NaN,NaN,NaN,2022-02-16,windy,jam,...,AGR,16,2,wednesday,0,15.0,19.0,evening,NaN,NaN
45499,CHENRES08DEL03,30.0,4.9,13.022394,80.242439,13.052394,80.272439,2022-03-11,cloudy,low,...,CHEN,11,3,friday,0,15.0,23.0,night,4.657195,short
45500,COIMBRES11DEL01,20.0,4.7,11.001753,76.986241,11.041753,77.026241,2022-03-07,cloudy,high,...,COIMB,7,3,monday,0,5.0,13.0,afternoon,6.232393,medium


In [15]:
swiggy['order_time_hour'].describe()

,order_time_hour
count,43862.000000
mean,17.423966
std,4.817856
min,0.000000
25%,15.000000
50%,19.000000
75%,21.000000
max,23.000000


In [16]:
def bin_time(df:pd.DataFrame,num_col):
    time_labels=['after_midnight','morning','afternoon','evening','night']
    time_bins=[0,6,12,17,20,24]
    df['order_time_bins']=pd.cut(df[num_col],bins=time_bins,labels=time_labels,right=False)
    return df

In [17]:
bin_time(swiggy,'order_time_hour')

,rider_id,age,ratings,restaurant_latitude,restaurant_longitude,delivery_latitude,delivery_longitude,order_date,weather,traffic,...,order_day,order_month,order_day_of_week,is_weekend,pickup_time_minutes,order_time_hour,order_time_of_day,distance,distance_bins,order_time_bins
0,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,sunny,high,...,19,3,saturday,1,15.0,11.0,morning,3.025149,short,morning
1,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,stormy,jam,...,25,3,friday,0,5.0,19.0,evening,20.183530,very long,evening
2,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,sandstorms,low,...,19,3,saturday,1,15.0,8.0,morning,1.552758,short,morning
3,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,sunny,medium,...,5,4,tuesday,0,10.0,18.0,evening,7.790401,medium,evening
4,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,cloudy,high,...,26,3,saturday,1,15.0,13.0,afternoon,6.210138,medium,afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45497,JAPRES04DEL01,30.0,4.8,26.902328,75.794257,26.912328,75.804257,2022-03-24,windy,high,...,24,3,thursday,0,10.0,11.0,morning,1.489846,short,morning
45498,AGRRES16DEL01,21.0,4.6,NaN,NaN,NaN,NaN,2022-02-16,windy,jam,...,16,2,wednesday,0,15.0,19.0,evening,NaN,NaN,evening
45499,CHENRES08DEL03,30.0,4.9,13.022394,80.242439,13.052394,80.272439,2022-03-11,cloudy,low,...,11,3,friday,0,15.0,23.0,night,4.657195,short,night
45500,COIMBRES11DEL01,20.0,4.7,11.001753,76.986241,11.041753,77.026241,2022-03-07,cloudy,high,...,7,3,monday,0,5.0,13.0,afternoon,6.232393,medium,afternoon


In [18]:
df1=swiggy.copy().dropna()
df1

,rider_id,age,ratings,restaurant_latitude,restaurant_longitude,delivery_latitude,delivery_longitude,order_date,weather,traffic,...,order_day,order_month,order_day_of_week,is_weekend,pickup_time_minutes,order_time_hour,order_time_of_day,distance,distance_bins,order_time_bins
0,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,sunny,high,...,19,3,saturday,1,15.0,11.0,morning,3.025149,short,morning
1,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,stormy,jam,...,25,3,friday,0,5.0,19.0,evening,20.183530,very long,evening
2,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,sandstorms,low,...,19,3,saturday,1,15.0,8.0,morning,1.552758,short,morning
3,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,sunny,medium,...,5,4,tuesday,0,10.0,18.0,evening,7.790401,medium,evening
4,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,cloudy,high,...,26,3,saturday,1,15.0,13.0,afternoon,6.210138,medium,afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45496,RANCHIRES16DEL01,35.0,4.2,23.371292,85.327872,23.481292,85.437872,2022-03-08,windy,jam,...,8,3,tuesday,0,10.0,21.0,night,16.600272,long,night
45497,JAPRES04DEL01,30.0,4.8,26.902328,75.794257,26.912328,75.804257,2022-03-24,windy,high,...,24,3,thursday,0,10.0,11.0,morning,1.489846,short,morning
45499,CHENRES08DEL03,30.0,4.9,13.022394,80.242439,13.052394,80.272439,2022-03-11,cloudy,low,...,11,3,friday,0,15.0,23.0,night,4.657195,short,night
45500,COIMBRES11DEL01,20.0,4.7,11.001753,76.986241,11.041753,77.026241,2022-03-07,cloudy,high,...,7,3,monday,0,5.0,13.0,afternoon,6.232393,medium,afternoon


In [19]:
columns_to_drop=[
    'rider_id',
    'restaurant_latitude',
    'restaurant_longitude',
    'delivery_latitude',
    'delivery_longitude',
    'order_date',
    'order_time_of_day',
    "order_time_hour",
    "order_day",
    "city_name",
    "order_day_of_week",
    "order_month"
]

In [20]:
df1.drop(columns=columns_to_drop,inplace=True)

In [21]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38064 entries, 0 to 45501
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   age                  38064 non-null  float64 
 1   ratings              38064 non-null  float64 
 2   weather              38064 non-null  object  
 3   traffic              38064 non-null  object  
 4   vehicle_condition    38064 non-null  int64   
 5   type_of_order        38064 non-null  object  
 6   type_of_vehicle      38064 non-null  object  
 7   multiple_deliveries  38064 non-null  float64 
 8   festival             38064 non-null  object  
 9   city_type            38064 non-null  object  
 10  time_taken           38064 non-null  int64   
 11  is_weekend           38064 non-null  int64   
 12  pickup_time_minutes  38064 non-null  float64 
 13  distance             38064 non-null  float64 
 14  distance_bins        38064 non-null  category
 15  order_time_bins      380

In [22]:
df1.isnull().mean()

,0
age,0.0
ratings,0.0
weather,0.0
traffic,0.0
vehicle_condition,0.0
type_of_order,0.0
type_of_vehicle,0.0
multiple_deliveries,0.0
festival,0.0
city_type,0.0


In [23]:
df1.duplicated().sum()

np.int64(0)

In [24]:
X=df1.drop(columns=['time_taken'])
X

,age,ratings,weather,traffic,vehicle_condition,type_of_order,type_of_vehicle,multiple_deliveries,festival,city_type,is_weekend,pickup_time_minutes,distance,distance_bins,order_time_bins
0,37.0,4.9,sunny,high,2,snack,motorcycle,0.0,no,urban,1,15.0,3.025149,short,morning
1,34.0,4.5,stormy,jam,2,snack,scooter,1.0,no,metropolitian,0,5.0,20.183530,very long,evening
2,23.0,4.4,sandstorms,low,0,drinks,motorcycle,1.0,no,urban,1,15.0,1.552758,short,morning
3,38.0,4.7,sunny,medium,0,buffet,motorcycle,1.0,no,metropolitian,0,10.0,7.790401,medium,evening
4,32.0,4.6,cloudy,high,1,snack,scooter,1.0,no,metropolitian,1,15.0,6.210138,medium,afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45496,35.0,4.2,windy,jam,2,drinks,motorcycle,1.0,no,metropolitian,0,10.0,16.600272,long,night
45497,30.0,4.8,windy,high,1,meal,motorcycle,0.0,no,metropolitian,0,10.0,1.489846,short,morning
45499,30.0,4.9,cloudy,low,1,drinks,scooter,0.0,no,metropolitian,0,15.0,4.657195,short,night
45500,20.0,4.7,cloudy,high,0,snack,motorcycle,1.0,no,metropolitian,0,5.0,6.232393,medium,afternoon


In [25]:
Y=df1['time_taken']
Y

,time_taken
0,24
1,33
2,26
3,21
4,30
...,...
45496,33
45497,32
45499,16
45500,26


In [26]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=10)

In [27]:
print("The size of train data is",X_train.shape)
print("The shape of test data is",X_test.shape)

The size of train data is (30451, 15)
The shape of test data is (7613, 15)


In [28]:
numerical_col=[cols for cols in df1.columns if df1[cols].dtype in ['int64','float64'] and cols !='time_taken']
numerical_col

['age',
 'ratings',
 'vehicle_condition',
 'multiple_deliveries',
 'is_weekend',
 'pickup_time_minutes',
 'distance']

In [29]:
categorical_col=[cols for cols in df1.columns if df1[cols].dtype in ['object','category']]
categorical_col

['weather',
 'traffic',
 'type_of_order',
 'type_of_vehicle',
 'festival',
 'city_type',
 'distance_bins',
 'order_time_bins']

In [30]:
nominal_col=['weather','type_of_order','type_of_vehicle','festival','distance_bins','order_time_bins']
ordinal_col=['traffic','city_type']

In [31]:
len(numerical_col+nominal_col+ordinal_col)

15

In [32]:
df1['traffic'].unique()

array(['high', 'jam', 'low', 'medium'], dtype=object)

In [33]:
df1['city_type'].unique()

array(['urban', 'metropolitian', 'semi-urban'], dtype=object)

In [34]:
traffic_groups=['low','medium','high','jam']
city_groups=['semi-urban','urban','metropolitian']

In [35]:
preprocessor=ColumnTransformer(
    [
        ("scaler",StandardScaler(),numerical_col),
        ("ohe",OneHotEncoder(drop="first",sparse_output=False,handle_unknown="ignore"),nominal_col),
        ("ordinal",OrdinalEncoder(categories=[traffic_groups,city_groups]),ordinal_col)
    ],remainder='passthrough'
)
preprocessor.set_output(transform="pandas")

ColumnTransformer(remainder='passthrough',
                  transformers=[('scaler', StandardScaler(),
                                 ['age', 'ratings', 'vehicle_condition',
                                  'multiple_deliveries', 'is_weekend',
                                  'pickup_time_minutes', 'distance']),
                                ('ohe',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 ['weather', 'type_of_order', 'type_of_vehicle',
                                  'festival', 'distance_bins',
                                  'order_time_bins']),
                                ('ordinal',
                                 OrdinalEncoder(categories=[['low', 'medium',
                                                             'high', 'jam'],
                                                            ['semi-urban',
                                                             'urban',
                                                             'metropolitian']]),
                                 ['traffic', 'city_type'])])

In [36]:
base_model=RandomForestRegressor()

In [37]:
pipeline1=Pipeline([
    ("preprocessor",preprocessor),
    ("base_model",base_model)
])

In [38]:
reg=TransformedTargetRegressor(
    regressor=pipeline1,
    transformer=PowerTransformer()
)

In [39]:
reg.fit(X_train,Y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but PowerTransformer was fitted without feature names
  warnings.warn(


TransformedTargetRegressor(regressor=Pipeline(steps=[('preprocessor',
                                                      ColumnTransformer(remainder='passthrough',
                                                                        transformers=[('scaler',
                                                                                       StandardScaler(),
                                                                                       ['age',
                                                                                        'ratings',
                                                                                        'vehicle_condition',
                                                                                        'multiple_deliveries',
                                                                                        'is_weekend',
                                                                                        'pickup_time_minutes',
                                                                                        'distance']),
                                                                                      ('ohe',
                                                                                       OneHotEncoder(drop='first',
                                                                                                     handle_unknown='ignore',
                                                                                                     sparse_output=False),
                                                                                       ['weather',
                                                                                        'type_of_order',
                                                                                        'type_of_vehicle',
                                                                                        'festival',
                                                                                        'distance_bins',
                                                                                        'order_time_bins']),
                                                                                      ('ordinal',
                                                                                       OrdinalEncoder(categories=[['low',
                                                                                                                   'medium',
                                                                                                                   'high',
                                                                                                                   'jam'],
                                                                                                                  ['semi-urban',
                                                                                                                   'urban',
                                                                                                                   'metropolitian']]),
                                                                                       ['traffic',
                                                                                        'city_type'])])),
                                                     ('base_model',
                                                      RandomForestRegressor())]),
                           transformer=PowerTransformer())

In [40]:
y_pred_test=reg.predict(X_test)
y_pred_train=reg.predict(X_train)

In [41]:
print(f"The train mean absolute error is:{round(mean_absolute_error(Y_train,y_pred_train),2)} min")
print(f"The test mean absolute error is:{round(mean_absolute_error(Y_test,y_pred_test),2)} min")

The train mean absolute error is:1.15 min
The test mean absolute error is:3.14 min


In [42]:
print(f"The train r2 score is:{round(r2_score(Y_train,y_pred_train),2)}")
print(f"The test r2 score is:{round(r2_score(Y_test,y_pred_test),2)}")

The train r2 score is:0.98
The test r2 score is:0.82


In [43]:
pipeline2=Pipeline([
    ("preprocessor",preprocessor),
])

In [44]:
X_train_trans=pipeline2.fit_transform(X_train)
X_test_trans=pipeline2.transform(X_test)

In [45]:
X_train_trans

,scaler__age,scaler__ratings,scaler__vehicle_condition,scaler__multiple_deliveries,scaler__is_weekend,scaler__pickup_time_minutes,scaler__distance,ohe__weather_fog,ohe__weather_sandstorms,ohe__weather_stormy,...,ohe__festival_yes,ohe__distance_bins_medium,ohe__distance_bins_short,ohe__distance_bins_very long,ohe__order_time_bins_afternoon,ohe__order_time_bins_evening,ohe__order_time_bins_morning,ohe__order_time_bins_night,ordinal__traffic,ordinal__city_type
42620,-1.319586,0.528127,0.002935,0.437621,-0.615678,0.006393,0.198212,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,2.0
6159,0.936057,-0.104873,-1.221503,0.437621,-0.615678,1.230791,-1.468722,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0
18429,0.415524,0.844627,1.227374,-1.310051,1.624227,0.006393,-0.933730,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,2.0
2206,-1.319586,0.211627,1.227374,0.437621,-0.615678,0.006393,-1.199370,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0
11565,0.762546,-0.737873,0.002935,-1.310051,-0.615678,1.230791,-0.434219,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12203,-0.278520,0.211627,1.227374,0.437621,1.624227,0.006393,-0.633418,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0
11198,-0.452031,1.161127,0.002935,0.437621,-0.615678,1.230791,-0.078964,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,2.0
33459,1.109568,-2.636872,1.227374,2.185294,1.624227,-1.218005,1.861649,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,1.0
34862,-0.625542,0.844627,0.002935,-1.310051,1.624227,-1.218005,-1.210381,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


#### Hyperparameter Tuning:

In [55]:
def objective(trial):
  with mlflow.start_run(nested=True):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'max_depth': trial.suggest_int('max_depth', 5, 40),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 100),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
        'verbosity': -1,
        'n_jobs': -1
      }

    mlflow.log_params(params)

    LGBM=LGBMRegressor(**params)

    model=TransformedTargetRegressor(
        regressor=LGBM,
        transformer=PowerTransformer("yeo-johnson")
    )

    model.fit(X_train_trans,Y_train)
    y_pred=model.predict(X_test_trans)
    error=mean_absolute_error(Y_test,y_pred)
    scores = cross_val_score(model,
                      X_train_trans,
                      Y_train,
                      scoring="neg_mean_absolute_error",
                      cv=5,n_jobs=-1).mean()

    mlflow.log_metric("cross_val_score",-scores)


    return error

In [57]:
study=optuna.create_study(direction='minimize')
with mlflow.start_run(run_name="best_model"):
    study.optimize(objective,n_trials=30,n_jobs=-1)
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_score",study.best_value)

    LGBM=LGBMRegressor(**study.best_params)

    model=TransformedTargetRegressor(
        regressor=LGBM,
        transformer=PowerTransformer("yeo-johnson")

    )


    model.fit(X_train_trans,Y_train)
    y_pred_test=model.predict(X_test_trans)
    y_pred_train=model.predict(X_train_trans)
    error=mean_absolute_error(Y_test,y_pred_test)

    scores = cross_val_score(model,
                         X_train_trans,
                         Y_train,
                         scoring="neg_mean_absolute_error",
                         cv=5,n_jobs=-1)

    #Log metrics:
    mlflow.log_metric("training_error",mean_absolute_error(Y_train,y_pred_train))
    mlflow.log_metric("testing_error",mean_absolute_error(Y_test,y_pred_test))
    mlflow.log_metric("training_r2",r2_score(Y_train,y_pred_train))
    mlflow.log_metric("testing_r2",r2_score(Y_test,y_pred_test))
    mlflow.log_metric("cross_val_score",-scores.mean())

    #Log the best model:
    mlflow.sklearn.save_model(model, "model")
    mlflow.log_artifact("model")

[I 2025-09-22 07:45:11,356] A new study created in memory with name: no-name-14ce07c6-65af-45bd-8059-68818dfac088
[I 2025-09-22 07:45:36,583] Trial 0 finished with value: 3.4824566115245785 and parameters: {'learning_rate': 0.06718219307693084, 'num_leaves': 28, 'feature_fraction': 0.4974291801658709, 'bagging_fraction': 0.6522390815057753, 'bagging_freq': 6, 'max_depth': 27, 'min_data_in_leaf': 80, 'lambda_l1': 4.935896440500202, 'lambda_l2': 3.916551045574198}. Best is trial 0 with value: 3.4824566115245785.


🏃 View run legendary-smelt-684 at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/2/runs/c37ba96aba3b473db646f5bff972fa46
🧪 View experiment at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/2
🏃 View run worried-whale-698 at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/2/runs/7c86941a864e482b9ff6797dd750847d
🧪 View experiment at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/2


[I 2025-09-22 07:45:37,107] Trial 1 finished with value: 6.348673115102212 and parameters: {'learning_rate': 0.002621232300258508, 'num_leaves': 71, 'feature_fraction': 0.9029520222314068, 'bagging_fraction': 0.6118762133750049, 'bagging_freq': 8, 'max_depth': 14, 'min_data_in_leaf': 26, 'lambda_l1': 0.5002967466641045, 'lambda_l2': 4.7678408893638}. Best is trial 0 with value: 3.4824566115245785.


🏃 View run burly-swan-754 at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/2/runs/f2323eedafae4926b57a3a9ff8c4a80b
🧪 View experiment at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/2


[I 2025-09-22 07:45:55,143] Trial 3 finished with value: 3.867247746256086 and parameters: {'learning_rate': 0.02217925558126655, 'num_leaves': 98, 'feature_fraction': 0.7947788481995524, 'bagging_fraction': 0.7049273638486449, 'bagging_freq': 9, 'max_depth': 5, 'min_data_in_leaf': 77, 'lambda_l1': 0.39873282244293873, 'lambda_l2': 3.5799180355474602}. Best is trial 0 with value: 3.4824566115245785.
[I 2025-09-22 07:45:55,582] Trial 2 finished with value: 5.970504763618302 and parameters: {'learning_rate': 0.006178851751916804, 'num_leaves': 94, 'feature_fraction': 0.4107563846161402, 'bagging_fraction': 0.9808100416932832, 'bagging_freq': 5, 'max_depth': 26, 'min_data_in_leaf': 79, 'lambda_l1': 1.0033258457982392, 'lambda_l2': 3.4739539864814173}. Best is trial 0 with value: 3.4824566115245785.


...

🏃 View run invincible-robin-508 at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/2/runs/f7eca0690d644ce28416eb4d1e7f3daa
🧪 View experiment at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/2
🏃 View run clean-cat-995 at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/2/runs/901dc7733a7b4dc48519a925163adaec
🧪 View experiment at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/2


[I 2025-09-22 07:50:10,417] Trial 26 finished with value: 3.1205404134838752 and parameters: {'learning_rate': 0.05005340133126704, 'num_leaves': 85, 'feature_fraction': 0.8199867240064896, 'bagging_fraction': 0.848412593097351, 'bagging_freq': 2, 'max_depth': 37, 'min_data_in_leaf': 93, 'lambda_l1': 0.7220984737822648, 'lambda_l2': 0.738806686538708}. Best is trial 25 with value: 3.0375036460191005.
[I 2025-09-22 07:50:36,697] Trial 29 finished with value: 3.1365158178315675 and parameters: {'learning_rate': 0.06105114483578484, 'num_leaves': 82, 'feature_fraction': 0.7207941530027897, 'bagging_fraction': 0.7081462205649711, 'bagging_freq': 5, 'max_depth': 36, 'min_data_in_leaf': 60, 'lambda_l1': 0.9614473110306236, 'lambda_l2': 1.3520509762460402}. Best is trial 25 with value: 3.0375036460191005.


🏃 View run honorable-steed-84 at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/2/runs/830073aac0de410688b42a909eb12421
🧪 View experiment at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/2
🏃 View run angry-finch-586 at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/2/runs/a71c58efd25f4456b3c7d59f375210e7
🧪 View experiment at: https://dagshub.com/Arej02/swiggy_delivery_time_prediction.mlflow/#/experiments/2


[I 2025-09-22 07:50:36,729] Trial 28 finished with value: 3.1295014521999365 and parameters: {'learning_rate': 0.05149427807965793, 'num_leaves': 85, 'feature_fraction': 0.804632906221111, 'bagging_fraction': 0.7047214582189544, 'bagging_freq': 3, 'max_depth': 37, 'min_data_in_leaf': 54, 'lambda_l1': 0.8282013014793469, 'lambda_l2': 1.2094681363921085}. Best is trial 25 with value: 3.0375036460191005.
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but PowerTransformer was fitted without feature names
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=62, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=62
[LightGBM] [Warning] feature_fraction is set=0.8475689756385575, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8475689756385575
[LightGBM] [Warning] lambda_l2 is set=0.6632645081461954, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.6632645081461954
[LightGBM] [Warning] lambda_l1 is set=0.0038672702290316607, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0038672702290316607
[LightGBM] [Warning] bagging_fraction is set=0.845652828843328, subsample=1.0 will be ignored. Current value: bagging_fraction=0.845652828843328
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_data_in_leaf is set=62, min_child_samples=20 will be ignored. Current value: min_data_in_leaf

In [58]:
study.best_params

{'learning_rate': 0.05597085845487942,
 'num_leaves': 85,
 'feature_fraction': 0.8475689756385575,
 'bagging_fraction': 0.845652828843328,
 'bagging_freq': 2,
 'max_depth': 36,
 'min_data_in_leaf': 62,
 'lambda_l1': 0.0038672702290316607,
 'lambda_l2': 0.6632645081461954}

In [59]:
study.best_value

3.0375036460191005